<a href="https://colab.research.google.com/github/ishahmshah1025/Mindspark-24/blob/main/twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install praw


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 8.2 MB/s eta 0:00:00


In [18]:
import praw
from textblob import TextBlob

# Set up Reddit API client
reddit = praw.Reddit(
    client_id='t2bNyMB4wU0yXdeWb7nVcQ',
    client_secret='Y5iOhI76laXgItwsSFTiKfDcRNyqwQ',
    user_agent='QuantumCoders/0.1 by Ok_profession1025'
)

subreddit = reddit.subreddit('stocks')

posts = subreddit.new(limit=10)

# Loop through each post
for post in posts:
    print(f"Title: {post.title}")
    print(f"Body: {post.selftext}")

    # Combine title and body for sentiment analysis
    text = post.title + " " + post.selftext

    # Perform sentiment analysis with TextBlob
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity

    # Classify sentiment based on polarity score
    if sentiment > 0:
        sentiment_label = "Positive"
    elif sentiment < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"

    # Print the sentiment result
    print(f"Sentiment: {sentiment_label}")
    print("-" * 40)


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Title: Now what? Hold? Gift?
Body: I bought NVDA at 6.75 in 2020 and am up 25K with what’s left. I’ve sold 2/3 of my original investment already on the way up. Do I hold the rest forever? I have an REIT that I’m down 10k on that I could use to offset this. All my other positions are up but nothing like this. I was considering buying first house soon and will have to sell off some positions for down payment but I wouldn’t have to touch this if I don’t want. Current tax burden of +3k this year outside of W2. I maxed all my 401k/IRA/HSA so my taxable income will be pretty low this year. Just looking for ideas. Gift it to a family member? 
Sentiment: Positive
----------------------------------------
Title: Nebius set to resume Nasdaq trading after completing split from Russia’s Yandex
Body: Oct 17 (Reuters) - AI infrastructure firm Nebius Group (NBIS.O), opens new tab said on Thursday its shares will resume Nasdaq trading on Monday after being halted following Russia's February 2022 invasi